E.T.L

Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

In [103]:
#Importo las librerias a utilizar.
import pandas as pd
import numpy as np
import json
from ast import literal_eval

In [104]:
#Utilizo la libreria Pandas para leer el archivo csv.
data = pd.read_csv('../movies_dataset.csv', sep=',', encoding='utf-8', low_memory = False)

--------------

Comienzo a examinar el dataset, evaluando los pasos a seguir.

In [105]:
#Visualizo las primeras 5 filas del dataset.
data.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [106]:
#Solicito la cantidad de filas y columnas. (Filas: 45466, Columnas:24).
data.shape

(45466, 24)

-------------------------

Comienzo a trabajar el dataset.

In [107]:
#Elimino las columnas que no voy a utilizar.
data.drop(['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'], axis=1, inplace=True)

In [108]:
#Solicito nuevamente la cantidad de filas y columnas para verificar que se hayan eliminado. (Filas: 45466, Columnas:18)
data.shape

(45466, 18)

In [109]:
data.head(3)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0
1,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0


In [110]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4494 non-null   object 
 1   budget                 45466 non-null  object 
 2   genres                 45466 non-null  object 
 3   id                     45466 non-null  object 
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45461 non-null  object 
 7   production_companies   45463 non-null  object 
 8   production_countries   45463 non-null  object 
 9   release_date           45379 non-null  object 
 10  revenue                45460 non-null  float64
 11  runtime                45203 non-null  float64
 12  spoken_languages       45460 non-null  object 
 13  status                 45379 non-null  object 
 14  tagline                20412 non-null  object 
 15  ti

----------------

Creo una funcion para visualizar todos los valores nulos del dataset.

In [111]:
# Valores nulos en el dataset
def have_nan(df):
    for col in df.columns:
        if df[col].isna().values.any():
            print(col, df[col].isna().values.sum())

have_nan(data)

belongs_to_collection 40972
original_language 11
overview 954
popularity 5
production_companies 3
production_countries 3
release_date 87
revenue 6
runtime 263
spoken_languages 6
status 87
tagline 25054
title 6
vote_average 6
vote_count 6


-----------------

Columnas anidadas, diccionarios y listas de diccionarios.

In [112]:
data['belongs_to_collection'].head()

0    {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                  NaN
2    {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                  NaN
4    {'id': 96871, 'name': 'Father of the Bride Col...
Name: belongs_to_collection, dtype: object

In [113]:
data['belongs_to_collection'].replace("'",'"',inplace=True)

In [114]:
# Reemplazo el formato de comillas en la columna 'belongs_to_collection'
data['belongs_to_collection'].replace("'",'"', inplace = True) 
# Los convierto en string
data['belongs_to_collection'] = data['belongs_to_collection'].astype(str)
subdicc = dict()

In [115]:
# Creo una funcion para recorrer el df y extraer los datos de la columna 'belongs_to_collection' 
for i in range(len(data)):
    franquicia = data.iloc[i]['belongs_to_collection']
    if pd.notna(franquicia):  
    
        try:
            subdicc = eval(franquicia)  # Uso 'eval' para convertir la cadena en un diccionario
            if isinstance(subdicc, dict):
                data.at[i, 'franquicia'] = subdicc.get('name')
            else:
                data.at[i, 'franquicia'] = None
        except:
            data.at[i, 'franquicia'] = None
    else:
        data.at[i, 'franquicia'] = None

data['franquicia']

0                  Toy Story Collection
1                                  None
2             Grumpy Old Men Collection
3                                  None
4        Father of the Bride Collection
                      ...              
45461                              None
45462                              None
45463                              None
45464                              None
45465                              None
Name: franquicia, Length: 45466, dtype: object

In [116]:
data.head(3)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,franquicia
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection
1,nan,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection


In [117]:
# Elimino la columna belongs_to_collection ya que no es relevante para el analisis.
data.drop(columns=['belongs_to_collection'],inplace=True)

In [118]:
data.head(3)

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,franquicia
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection
1,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None
2,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection


------------------

In [119]:
lista_productoras = []

for i in range(len(data)):
    lista_productoras = data.iloc[i]['production_companies']

    if pd.notna(lista_productoras):
    
        if isinstance(lista_productoras, str):
            # Si es una cadena, intentar cargarla como JSON
            try:
                lista_productoras = json.loads(lista_productoras.replace("'", "\""))
            except json.JSONDecodeError:
                lista_productoras = None
            
        if isinstance(lista_productoras, list):
            # Si es lista, procesar los diccionarios
            list_productora = []
            for valor in lista_productoras:
                if isinstance(valor, dict):
                    # si es un diccionario, obtener el nombre
                    productora_nombre = valor.get('name')
                    if productora_nombre:
                        list_productora.append(productora_nombre)
                elif isinstance(valor, str):
                    try:
                        # Si es una cadena JSON, cargarla como diccionario
                        diccionario = json.loads(valor.replace("'", "\""))
                        productora_nombre = diccionario.get('name')
                        if productora_nombre:
                            list_productora.append(productora_nombre)
                    except json.JSONDecodeError:
                        pass

            data.at[i, 'productoras'] = list_productora
        else:
            data.at[i, 'productoras'] = None
    else:
        data.at[i, 'productoras'] = None


In [120]:
# Creo una funcion para que elimine los corchetes de la lista.
def eliminar_corchetes(lista):
    if isinstance(lista, list):
        return ', '.join(lista)  # Se une los elementos de la lista con una coma y espacio
    return lista

In [121]:
data['productoras'] = data['productoras'].apply(eliminar_corchetes)

In [122]:
data['productoras']

0                                  Pixar Animation Studios
1        TriStar Pictures, Teitler Film, Interscope Com...
2                             Warner Bros., Lancaster Gate
3                   Twentieth Century Fox Film Corporation
4               Sandollar Productions, Touchstone Pictures
                               ...                        
45461                                                     
45462                                          Sine Olivia
45463                              American World Pictures
45464                                            Yermoliev
45465                                                     
Name: productoras, Length: 45466, dtype: object

In [123]:
# Realizo lo mismo para la columna 'production_countries'.
lista_paises = []

for i in range(len(data)):
    lista_paises = data.iloc[i]['production_countries']

    if pd.notna(lista_paises):
    
        if isinstance(lista_paises, str):
            try:
                lista_paises = json.loads(lista_paises.replace("'", "\""))
            except json.JSONDecodeError:
                lista_paises = None
            
        if isinstance(lista_paises, list):
            paises = []
            for valor in lista_paises:
                if isinstance(valor, dict):
                    nombre = valor.get('name')
                    if nombre:
                        paises.append(nombre)
                elif isinstance(valor, str):
                    try:
                        diccionario = json.loads(valor.replace("'", "\""))
                        nombre = diccionario.get('name')
                        if nombre:
                            paises.append(nombre)
                    except json.JSONDecodeError:
                        pass

            data.at[i, 'paises'] = paises
        else:
            data.at[i, 'paises'] = None
    else:
        data.at[i, 'paises'] = None


In [124]:
data['paises'] = data['paises'].apply(eliminar_corchetes)

In [125]:
data['paises']

0        United States of America
1        United States of America
2        United States of America
3        United States of America
4        United States of America
                   ...           
45461                        Iran
45462                 Philippines
45463    United States of America
45464                      Russia
45465              United Kingdom
Name: paises, Length: 45466, dtype: object

----------------------------------------

--------

In [126]:
data.head(1)

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,franquicia,productoras,paises
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,Pixar Animation Studios,United States of America


--------------------------

In [127]:
# Visualizo la columna 'original_languages' que sera necesaria para las consultas en la API.
data['original_language'].isna().sum()

11

In [128]:
# trato los valores nulos en la columna 'original_language' con el valor 'unknown' para no eliminar la fila completa.
data['original_language'].fillna('unknown', inplace=True)

In [129]:
# Corroboro que se haya aplicado el reemplazo.
data['original_language'].isnull().sum()

0

--------------------------

Verifico el tipo de dato de 'runtime' y 'release_year' para utilizarlos luego en la API, normalizo el tipo de datos para ser consumidos por la api.

In [130]:
data['runtime'].head(3)

0     81.0
1    104.0
2    101.0
Name: runtime, dtype: float64

In [131]:
data['runtime'].fillna(0, inplace=True)

In [132]:
# Decido cambiar el tipo de dato de la columna 'runtime' de float a int, quedando en minutos.
data['runtime'] = data['runtime'].astype(int)


In [133]:
data['runtime'].head()

0     81
1    104
2    101
3    127
4    106
Name: runtime, dtype: int32

In [134]:
data.head(3)

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,franquicia,productoras,paises
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,Pixar Animation Studios,United States of America
1,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America
2,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,"Warner Bros., Lancaster Gate",United States of America


----------

Trabajo con la columna 'release_date'. Normalizo los datos, y creo una nueva columna llamada 'release_year'.

In [135]:
# Convierto 'release_date' a formato datetime con Pandas, errors='coerce' para que los valores que no se puedan convertir a datetime se conviertan en NaN y no generen un error
data['release_date'] = pd.to_datetime(data['release_date'], errors='coerce')

In [136]:
#Elimino las filas de 'release_date' que tengan valores NaN
data.dropna(subset=['release_date'], inplace=True)
#Inserto una nueva columna llamada 'release_year' en el dataframe luego de release_date y luego asigno los valores correspondientes a la nueva columna
data.insert(data.columns.get_loc('release_date') + 1, 'release_year', None)
data['release_year'] = data['release_date'].dt.year

In [137]:
# Creo una funcion listOfColumns que recorra las columnas ingresadas y reemplace los valores NaN por 0
def replace_to_cero(listOfColumns: list[str]):
    for col in listOfColumns:
            data[col].fillna(0, inplace=True)

listOfColumns = ['id','budget','revenue','runtime','popularity','vote_average','vote_count']
replace_to_cero(listOfColumns)

In [138]:
data['release_date'].head(3), data['release_year'].head(3)

(0   1995-10-30
 1   1995-12-15
 2   1995-12-22
 Name: release_date, dtype: datetime64[ns],
 0    1995
 1    1995
 2    1995
 Name: release_year, dtype: int64)

In [139]:
data['release_year']

0        1995
1        1995
2        1995
3        1995
4        1995
         ... 
45460    1991
45462    2011
45463    2003
45464    1917
45465    2017
Name: release_year, Length: 45376, dtype: int64

------------------------


Trabajo con las columnas 'revenue' y 'budget', normalizo los datos y realizo la operacion para calcular el retorno de inversion creando una nueva columna llamada 'return'. 

In [140]:
# Transformo ambas columnas a numerico.
data['revenue'] = pd.to_numeric(data['revenue'], errors='coerce')
data['budget'] = pd.to_numeric(data['budget'], errors='coerce')

# Creo una nueva columna 'return' que contenga el resultado de la división de 'revenue' y 'budget'.
data['return'] = data['revenue'] / data['budget']


In [141]:
# Solicito que me muestre la columna creada, observo valores NaN e infinitos por lo que trabajare con la columna 'return' para normalizarlo.
data['return']

0        12.451801
1         4.043035
2              NaN
3         5.090760
4              inf
           ...    
45460          NaN
45462          NaN
45463          NaN
45464          NaN
45465          NaN
Name: return, Length: 45376, dtype: float64

In [142]:
data['return'].replace(float('inf'), 0, inplace=True)

In [143]:
#Utilizo la funcion creada en la columna 'return' para reemplazar los valores NaN por 0
replace_to_cero(['return'])

In [144]:
data['return'].head()

0    12.451801
1     4.043035
2     0.000000
3     5.090760
4     0.000000
Name: return, dtype: float64

In [145]:
#Redondeo los valores de la columna 'return' a 2 decimales
data['return'] = data['return'].apply(lambda x: round(x, 2))

In [146]:
data['return'].head(5)

0    12.45
1     4.04
2     0.00
3     5.09
4     0.00
Name: return, dtype: float64

In [147]:
# Guardo el dataset en un nuevo archivo csv.
data.to_csv('../movies_clean.csv', index=False)